In [2]:
#initialize code and import packages
import dask
import os
from dask.distributed import LocalCluster, Client, wait
import os
import convenient_bgc as cv
from mapping_functions_bgc import *
import xarray as xr
import numpy as np

In [ ]:
##FOR USE ON SCRATCH
scheduler_file = os.path.join(os.environ["SCRATCH"], "scheduler_file.json")

dask.config.config["distributed"]["dashboard"]["link"] = "{JUPYTERHUB_SERVICE_PREFIX}proxy/{host}:{port}/status" 

client = Client(scheduler_file=scheduler_file)
client

In [ ]:
client.upload_file("njit_options.py")
client.upload_file("basisfunctions_bgc.py")
client.upload_file("distancefunctions.py")
client.upload_file("mean_field_lsq_bgc.py")

In [3]:
#TESTING ON HOMES ONLY
cluster = LocalCluster()  # Launches a scheduler and workers locally
client = Client(cluster)  # Connect to distributed cluster and override default
print('External dashboard-link: ', client.dashboard_link)
print(client)

/global/homes/k/kefalc/.conda/envs/myenv/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39687 instead
  warnings.warn(


External dashboard-link:  http://127.0.0.1:39687/status
<Client: 'tcp://127.0.0.1:33737' processes=16 threads=256, memory=476.37 GiB>


In [6]:
window_size_betaparam = math.sqrt(1.0 * 700e3**2)
NHarm = 2
testn = 0
datafield = 'oxygen'
bgcdatafile = '/global/homes/k/kefalc/code/data_files/Final/floats_years_2015_2019_temperature_salinity_oxygen_10daily_SO_30Skeep_random_floats_15perc.nc'
maskfile = '/global/homes/k/kefalc/code/gpr-mapping-mean/SO_mask.nc'

time_i = (np.datetime64('2015-01-01') - np.datetime64('1970-01-01T00:00:00', 'ns')) / np.timedelta64(1, 'ns') / 1e9 / 86400.0
time_f = (np.datetime64('2019-12-31') - np.datetime64('1970-01-01T00:00:00', 'ns')) / np.timedelta64(1, 'ns') / 1e9 / 86400.0

grid_time_i = (np.datetime64('2017-01-01') - np.datetime64('1970-01-01T00:00:00', 'ns')) / np.timedelta64(1, 'ns') / 1e9 / 86400.0
grid_time_f = (np.datetime64('2017-12-31') - np.datetime64('1970-01-01T00:00:00', 'ns')) / np.timedelta64(1, 'ns') / 1e9 / 86400.0


In [7]:
%%time

for plevel in range(1,7): # Select pressure levels
    #input files
    tsmeanfile = '/global/homes/k/kefalc/code/data_files/Final'+'/forbgc_residualfile'+'_plevel'+str(plevel)+'_dates'+str(time_i)+'_'+str(time_f)+'_test'+str(testn)+'.nc'
    tsgridfile = '/global/homes/k/kefalc/code/anom_files/test'+'/grid_TS_mean_ws'+str(np.int64(window_size_betaparam/1e3))+'_plevel'+str(plevel)+'_dates'+str(grid_time_i)+'_'+str(grid_time_f)+'_test'+str(testn)+'.nc'
    
    #files to be saved
    betaparamfile= '/global/homes/k/kefalc/code/anom_files/'+str(datafield)+'_betaparam_plevel'+str(plevel)+'_dates'+str(time_i)+'_'+str(time_f)+'_test'+str(testn)+'.nc'
    residualfile = '/global/homes/k/kefalc/code/anom_files/test'+'/residualfile_plevel'+str(plevel)+str(datafield)+'_dates'+str(time_i)+'_'+str(time_f)+'_test'+str(testn)+'.nc'
    bgc_gridmeanfile = '/global/homes/k/kefalc/code/anom_files/test'+'/grid_'+str(datafield)+'_mean_ws'+str(np.int64(window_size_betaparam/1e3))+'_plevel'+str(plevel)+'_dates'+str(grid_time_i)+'_'+str(grid_time_f)+'_test'+str(testn)+'.nc'
    
    #functions
    bgc_betaparam( datafield, plevel, bgcdatafile, tsmeanfile, betaparamfile, maskfile, window_size_betaparam, NHarm, client)
    interpolate_to_observations_bgc( datafield, bgcdatafile, betaparamfile, tsmeanfile, residualfile, plevel ) 
    grid_mean_seasonal_BGC( plevel, datafield, maskfile, betaparamfile, tsgridfile, bgc_gridmeanfile, bgcdatafile, grid_time_i, grid_time_f, window_size_betaparam )


-10 dbar pressurelevel (pressureindex 1): Betaparameters saved to outfile.
  Interpolated to 36506 coordinates. Nan-estimate: 0.0. Mean and residuals calculated and saved to outfile.
  Interpolated to 14346 coordinates. Nan-estimate: 0.0.

/global/homes/k/kefalc/.conda/envs/myenv/lib/python3.11/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 12.43 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


 Gridded estimated 5-day mean saved to outfile.
Betaparameters saved to outfile.eindex 2): 
  Interpolated to 36506 coordinates. Nan-estimate: 0.0. Mean and residuals calculated and saved to outfile.
  Interpolated to 14346 coordinates. Nan-estimate: 0.0.

/global/homes/k/kefalc/.conda/envs/myenv/lib/python3.11/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 12.43 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


 Gridded estimated 5-day mean saved to outfile.
Betaparameters saved to outfile.eindex 3): 
  Interpolated to 36506 coordinates. Nan-estimate: 0.0. Mean and residuals calculated and saved to outfile.
  Interpolated to 14346 coordinates. Nan-estimate: 0.0.

...

/global/homes/k/kefalc/.conda/envs/myenv/lib/python3.11/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 12.43 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


 Gridded estimated 5-day mean saved to outfile.
Betaparameters saved to outfile.reindex 5): 
  Interpolated to 36506 coordinates. Nan-estimate: 0.0. Mean and residuals calculated and saved to outfile.
  Interpolated to 14346 coordinates. Nan-estimate: 0.0.

/global/homes/k/kefalc/.conda/envs/myenv/lib/python3.11/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 12.43 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


 Gridded estimated 5-day mean saved to outfile.
Betaparameters saved to outfile.reindex 6): 
  Interpolated to 36506 coordinates. Nan-estimate: 0.0. Mean and residuals calculated and saved to outfile.
  Interpolated to 14346 coordinates. Nan-estimate: 0.0. Gridded estimated 5-day mean saved to outfile.
CPU times: user 1h 18min 45s, sys: 5min 5s, total: 1h 23min 50s
Wall time: 1h 18min 36s


/global/homes/k/kefalc/.conda/envs/myenv/lib/python3.11/site-packages/distributed/client.py:3162: UserWarning: Sending large graph of size 12.43 MiB.
This may cause some slowdown.
Consider scattering data ahead of time and using futures.
  warnings.warn(


In [ ]:
cluster.shutdown()